In [1]:
import sys
sys.path.append("../")

In [126]:
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
from technicals.indicators import RSI
from technicals.patterns import apply_patterns
from plotting import CandlePlot

In [83]:
df_raw = pd.read_pickle("../data/EUR_USD_H1.pkl")

In [84]:
df_raw.shape

(44055, 14)

In [85]:
df_an = df_raw.copy() #df_raw.iloc[-6000:].copy()
df_an.reset_index(drop=True, inplace=True)

In [86]:
df_an.shape

(44055, 14)

In [87]:
df_an = RSI(df_an)

In [88]:
df_an.tail()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,RSI_14
44050,2023-01-31 19:00:00+00:00,2687,1.08722,1.08741,1.08630,1.08642,1.08715,1.08734,1.08622,1.08635,1.08728,1.08748,1.08637,1.08649,54.520710
44051,2023-01-31 20:00:00+00:00,3285,1.08643,1.08732,1.08599,1.08720,1.08636,1.08724,1.08592,1.08711,1.08650,1.08739,1.08606,1.08728,57.182766
44052,2023-01-31 21:00:00+00:00,1964,1.08720,1.08726,1.08618,1.08632,1.08712,1.08719,1.08610,1.08623,1.08727,1.08732,1.08626,1.08640,53.386083
44053,2023-01-31 22:00:00+00:00,547,1.08642,1.08646,1.08609,1.08618,1.08608,1.08627,1.08589,1.08598,1.08675,1.08677,1.08626,1.08637,52.785621
44054,2023-01-31 23:00:00+00:00,1334,1.08616,1.08657,1.08604,1.08608,1.08595,1.08650,1.08587,1.08599,1.08636,1.08666,1.08613,1.08618,52.332841


In [89]:
df_an = apply_patterns(df_an)

In [90]:
df_an['EMA_200'] = df_an.mid_c.ewm(span=200, min_periods=200).mean()

In [91]:
df_an.columns

Index(['time', 'volume', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_o', 'bid_h',
       'bid_l', 'bid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c', 'RSI_14',
       'body_lower', 'body_upper', 'body_bottom_perc', 'body_top_perc',
       'body_perc', 'direction', 'body_size', 'low_change', 'high_change',
       'body_size_change', 'mid_point', 'mid_point_prev_2', 'body_size_prev',
       'direction_prev', 'direction_prev_2', 'body_perc_prev',
       'body_perc_prev_2', 'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR', 'EMA_200'],
      dtype='object')

In [92]:
our_cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c',
            'ask_c', 'bid_c', 'ENGULFING', 'direction', 'EMA_200', 'RSI_14']

In [93]:
df_slim = df_an[our_cols].copy()
df_slim.dropna(inplace=True)
df_slim.reset_index(drop=True, inplace=True)

In [94]:
df_slim.head()

,time,mid_o,mid_h,mid_l,mid_c,ask_c,bid_c,ENGULFING,direction,EMA_200,RSI_14
0,2016-01-14 05:00:00+00:00,1.08914,1.08934,1.08763,1.08787,1.08795,1.08779,True,-1,1.085276,57.119596
1,2016-01-14 06:00:00+00:00,1.08784,1.08805,1.08669,1.08672,1.08680,1.08663,False,-1,1.085293,51.174264
2,2016-01-14 07:00:00+00:00,1.08672,1.08720,1.08544,1.08598,1.08606,1.08591,False,-1,1.085301,47.731446
3,2016-01-14 08:00:00+00:00,1.08594,1.09183,1.08567,1.09134,1.09141,1.09126,True,1,1.085370,65.720674
4,2016-01-14 09:00:00+00:00,1.09136,1.09366,1.08982,1.09022,1.09028,1.09015,False,-1,1.085425,60.996611


In [95]:
BUY = 1
SELL = -1
NONE = 0
RSI_LIMIT = 50.0

def apply_signal(row):
    if row.ENGULFING == True:
        if row.direction == BUY and row.mid_l > row.EMA_200:
            if row.RSI_14 > RSI_LIMIT:
                return BUY
        elif row.direction == SELL and row.mid_h < row.EMA_200:
            if row.RSI_14 < RSI_LIMIT:
                return SELL
    return NONE

In [96]:
df_slim['SIGNAL'] = df_slim.apply(apply_signal, axis=1)

In [97]:
df_slim['SIGNAL'].value_counts()

 0    39973
-1     1949
 1     1934
Name: SIGNAL, dtype: int64

In [98]:
LOSS_FACTOR = -1.0
PROFIT_FACTOR = 1.5

def apply_take_profit(row):
    if row.SIGNAL != NONE:
        return (row.mid_c - row.mid_o) * PROFIT_FACTOR + row.mid_c
    else:
        return 0.0
    
def apply_stop_loss(row):
    if row.SIGNAL != NONE:
        return row.mid_o
    else:
        return 0.0

In [99]:
df_slim['TP'] = df_slim.apply(apply_take_profit, axis=1)
df_slim['SL'] = df_slim.apply(apply_stop_loss, axis=1)

In [100]:
df_slim[df_slim.SIGNAL == BUY].head()

,time,mid_o,mid_h,mid_l,mid_c,ask_c,bid_c,ENGULFING,direction,EMA_200,RSI_14,SIGNAL,TP,SL
3,2016-01-14 08:00:00+00:00,1.08594,1.09183,1.08567,1.09134,1.09141,1.09126,True,1,1.085370,65.720674,1,1.099440,1.08594
5,2016-01-14 10:00:00+00:00,1.09019,1.09289,1.08996,1.09227,1.09233,1.09221,True,1,1.085504,65.837097,1,1.095390,1.09019
25,2016-01-15 06:00:00+00:00,1.08788,1.08898,1.08758,1.08872,1.08881,1.08864,True,1,1.085911,55.194357,1,1.089980,1.08788
27,2016-01-15 08:00:00+00:00,1.08685,1.09046,1.08634,1.09007,1.09014,1.09000,True,1,1.085968,58.204010,1,1.094900,1.08685
35,2016-01-15 16:00:00+00:00,1.09510,1.09680,1.09420,1.09635,1.09642,1.09628,True,1,1.086589,69.048533,1,1.098225,1.09510


In [101]:
df_plot = df_slim.iloc[140:220]
cp = CandlePlot(df_plot, candles=True)

trades = cp.df_plot[df_plot.SIGNAL != NONE]

markers = ['mid_c', 'TP', 'SL']
marker_colors = ['#0000FF', '#00FF00', '#FF0000']

for i in range(3):
    cp.fig.add_trace(go.Scatter(
        x = trades.sTime,
        y = trades[markers[i]],
        mode = 'markers',
        marker=dict(color=marker_colors[i], size=12)
    ))
    
cp.show_plot(line_traces=['EMA_200'], sec_traces=['RSI_14'], height=250)

In [102]:
df_results[df_results.SIGNAL == BUY].head()

,running,start_index,start_price,trigger_price,SIGNAL,TP,SL,result,end_time,start_time,duration
12,False,150,168.914,169.961,1,169.5485,168.491,1.5,2016-01-22 14:00:00+00:00,2016-01-22 11:00:00+00:00,3
13,False,153,169.932,168.996,1,171.1200,169.140,-1.0,2016-01-24 23:00:00+00:00,2016-01-22 14:00:00+00:00,9
14,False,169,169.875,169.158,1,170.1105,169.718,-1.0,2016-01-25 07:00:00+00:00,2016-01-25 06:00:00+00:00,1
15,False,175,169.218,168.538,1,170.1900,168.570,-1.0,2016-01-25 13:00:00+00:00,2016-01-25 12:00:00+00:00,1
18,False,215,169.733,169.026,1,170.2775,169.370,-1.0,2016-01-27 08:00:00+00:00,2016-01-27 04:00:00+00:00,4


In [103]:
class Trade:
    def __init__(self, row):
        self.running = True
        self.start_index = row.name
        self.start_price = row.mid_c
        self.trigger_price = row.mid_c
        self.SIGNAL = row.SIGNAL
        self.TP = row.TP
        self.SL = row.SL
        self.result = 0.0
        self.end_time = row.time
        self.start_time = row.time
        self.duration = 0
    
    def close_trade(self, row, result, trigger_price):
        self.running = False
        self.result = result
        self.end_time = row.time
        self.trigger_price = trigger_price
    
    def update(self, row):
        self.duration += 1
        if self.SIGNAL == BUY:
            if row.mid_h >= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.mid_h)
            elif row.mid_l <= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.mid_l)
        if self.SIGNAL == SELL:
            if row.mid_l <= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.mid_l)
            elif row.mid_h >= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.mid_h)
    

In [104]:
open_trades = []
closed_trades = []

for index, row in df_slim.iterrows():
    for ot in open_trades:
        ot.update(row)
        if ot.running == False:
            closed_trades.append(ot)
    open_trades = [x for x in open_trades if x.running == True]
    
    if row.SIGNAL != NONE:
        open_trades.append(Trade(row))

In [105]:
df_results = pd.DataFrame.from_dict([vars(x) for x in closed_trades])

In [106]:
df_results.result.sum()

332.0

In [107]:
df_results.sort_values(by="start_index", inplace=True)

In [110]:
df_m5 = pd.read_pickle("../data/EUR_USD_M5.pkl")

In [111]:
df_m5.shape

(526969, 14)

In [115]:
df_m5.time.max()

Timestamp('2023-01-31 23:55:00+0000', tz='tzutc()')

In [116]:
df_raw.time.max()

Timestamp('2023-01-31 23:55:00+0000', tz='tzutc()')

In [117]:
from dateutil import parser

In [118]:
time_min = parser.parse("2021-12-15T10:00:00Z")
time_max = parser.parse("2021-12-15T11:00:00Z")
df_m5_s = df_m5[(df_m5.time >= time_min) & (df_m5.time <= time_max)]
df_raw_s = df_raw[(df_raw.time >= time_min) & (df_raw.time <= time_max)]

In [119]:
df_m5_s

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
442607,2021-12-15 10:00:00+00:00,208,1.12716,1.12745,1.12709,1.12734,1.12708,1.12738,1.12702,1.12727,1.12723,1.12752,1.12715,1.12742
442608,2021-12-15 10:05:00+00:00,132,1.12733,1.12754,1.12726,1.12746,1.12726,1.12747,1.12719,1.12739,1.12740,1.12760,1.12733,1.12752
442609,2021-12-15 10:10:00+00:00,166,1.12744,1.12752,1.12732,1.12746,1.12737,1.12745,1.12725,1.12739,1.12750,1.12760,1.12739,1.12753
442610,2021-12-15 10:15:00+00:00,138,1.12744,1.12770,1.12740,1.12761,1.12737,1.12763,1.12733,1.12754,1.12752,1.12776,1.12747,1.12768
442611,2021-12-15 10:20:00+00:00,252,1.12760,1.12762,1.12712,1.12712,1.12753,1.12756,1.12705,1.12705,1.12767,1.12769,1.12719,1.12719
442612,2021-12-15 10:25:00+00:00,198,1.12711,1.12726,1.12695,1.12695,1.12704,1.12719,1.12688,1.12688,1.12718,1.12733,1.12702,1.12702
442613,2021-12-15 10:30:00+00:00,142,1.12696,1.12698,1.12680,1.12696,1.12689,1.12691,1.12674,1.12690,1.12703,1.12706,1.12686,1.12703
442614,2021-12-15 10:35:00+00:00,121,1.12698,1.12712,1.12698,1.12700,1.12691,1.12705,1.12691,1.12693,1.12704,1.12718,1.12704,1.12706
442615,2021-12-15 10:40:00+00:00,206,1.12698,1.12707,1.12651,1.12656,1.12692,1.12700,1.12644,1.12649,1.12705,1.12714,1.12658,1.12663
442616,2021-12-15 10:45:00+00:00,165,1.12655,1.12668,1.12645,1.12654,1.12648,1.12661,1.12638,1.12648,1.12662,1.12675,1.12652,1.12661


In [120]:
df_raw_s

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
442607,2021-12-15 10:00:00+00:00,208,1.12716,1.12745,1.12709,1.12734,1.12708,1.12738,1.12702,1.12727,1.12723,1.12752,1.12715,1.12742
442608,2021-12-15 10:05:00+00:00,132,1.12733,1.12754,1.12726,1.12746,1.12726,1.12747,1.12719,1.12739,1.12740,1.12760,1.12733,1.12752
442609,2021-12-15 10:10:00+00:00,166,1.12744,1.12752,1.12732,1.12746,1.12737,1.12745,1.12725,1.12739,1.12750,1.12760,1.12739,1.12753
442610,2021-12-15 10:15:00+00:00,138,1.12744,1.12770,1.12740,1.12761,1.12737,1.12763,1.12733,1.12754,1.12752,1.12776,1.12747,1.12768
442611,2021-12-15 10:20:00+00:00,252,1.12760,1.12762,1.12712,1.12712,1.12753,1.12756,1.12705,1.12705,1.12767,1.12769,1.12719,1.12719
442612,2021-12-15 10:25:00+00:00,198,1.12711,1.12726,1.12695,1.12695,1.12704,1.12719,1.12688,1.12688,1.12718,1.12733,1.12702,1.12702
442613,2021-12-15 10:30:00+00:00,142,1.12696,1.12698,1.12680,1.12696,1.12689,1.12691,1.12674,1.12690,1.12703,1.12706,1.12686,1.12703
442614,2021-12-15 10:35:00+00:00,121,1.12698,1.12712,1.12698,1.12700,1.12691,1.12705,1.12691,1.12693,1.12704,1.12718,1.12704,1.12706
442615,2021-12-15 10:40:00+00:00,206,1.12698,1.12707,1.12651,1.12656,1.12692,1.12700,1.12644,1.12649,1.12705,1.12714,1.12658,1.12663
442616,2021-12-15 10:45:00+00:00,165,1.12655,1.12668,1.12645,1.12654,1.12648,1.12661,1.12638,1.12648,1.12662,1.12675,1.12652,1.12661


In [123]:
df_m5_slim = df_m5[['time', 'mid_h', 'mid_l']].copy()

In [125]:
df_m5_slim.head()

,time,mid_h,mid_l
0,2016-01-03 22:00:00+00:00,1.08746,1.08724
1,2016-01-03 22:05:00+00:00,1.08738,1.08710
2,2016-01-03 22:10:00+00:00,1.08715,1.08674
3,2016-01-03 22:15:00+00:00,1.08694,1.08669
4,2016-01-03 22:20:00+00:00,1.08717,1.08670


In [138]:
df_signals = df_slim[df_slim.SIGNAL != NONE].copy()

In [139]:
df_signals['m5_start'] = [x + dt.timedelta(hours=1) for x in df_signals.time]

In [140]:
df_signals['start_index_h1'] = df_signals.index

In [141]:
df_signals.head()

,time,mid_o,mid_h,mid_l,mid_c,ask_c,bid_c,ENGULFING,direction,EMA_200,RSI_14,SIGNAL,TP,SL,m5_start,start_index_h1
3,2016-01-14 08:00:00+00:00,1.08594,1.09183,1.08567,1.09134,1.09141,1.09126,True,1,1.085370,65.720674,1,1.099440,1.08594,2016-01-14 09:00:00+00:00,3
5,2016-01-14 10:00:00+00:00,1.09019,1.09289,1.08996,1.09227,1.09233,1.09221,True,1,1.085504,65.837097,1,1.095390,1.09019,2016-01-14 11:00:00+00:00,5
25,2016-01-15 06:00:00+00:00,1.08788,1.08898,1.08758,1.08872,1.08881,1.08864,True,1,1.085911,55.194357,1,1.089980,1.08788,2016-01-15 07:00:00+00:00,25
27,2016-01-15 08:00:00+00:00,1.08685,1.09046,1.08634,1.09007,1.09014,1.09000,True,1,1.085968,58.204010,1,1.094900,1.08685,2016-01-15 09:00:00+00:00,27
35,2016-01-15 16:00:00+00:00,1.09510,1.09680,1.09420,1.09635,1.09642,1.09628,True,1,1.086589,69.048533,1,1.098225,1.09510,2016-01-15 17:00:00+00:00,35


In [142]:
df_signals.columns

Index(['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'ask_c', 'bid_c',
       'ENGULFING', 'direction', 'EMA_200', 'RSI_14', 'SIGNAL', 'TP', 'SL',
       'm5_start', 'start_index_h1'],
      dtype='object')

In [143]:
df_signals.drop(['time', 'mid_o', 'mid_h', 'mid_l', 'ask_c', 'bid_c', 
                 'ENGULFING', 'EMA_200', 'RSI_14', 'direction'], axis=1, inplace=True)

In [144]:
df_signals.head()

,mid_c,SIGNAL,TP,SL,m5_start,start_index_h1
3,1.09134,1,1.099440,1.08594,2016-01-14 09:00:00+00:00,3
5,1.09227,1,1.095390,1.09019,2016-01-14 11:00:00+00:00,5
25,1.08872,1,1.089980,1.08788,2016-01-15 07:00:00+00:00,25
27,1.09007,1,1.094900,1.08685,2016-01-15 09:00:00+00:00,27
35,1.09635,1,1.098225,1.09510,2016-01-15 17:00:00+00:00,35


In [145]:
df_signals.rename(columns={
    'mid_c': 'start_price',
    'm5_start': 'time'
}, inplace=True)

In [148]:
df_signals.head(2)

,start_price,SIGNAL,TP,SL,time,start_index_h1
3,1.09134,1,1.09944,1.08594,2016-01-14 09:00:00+00:00,3
5,1.09227,1,1.09539,1.09019,2016-01-14 11:00:00+00:00,5


In [147]:
df_m5_slim.head(2)

,time,mid_h,mid_l
0,2016-01-03 22:00:00+00:00,1.08746,1.08724
1,2016-01-03 22:05:00+00:00,1.08738,1.08710


In [149]:
merged = pd.merge(left=df_m5_slim, right=df_signals, on='time', how='left')

In [152]:
merged.fillna(0, inplace=True)

In [156]:
merged.SIGNAL = merged.SIGNAL.astype(int)
merged.start_index_h1 = merged.start_index_h1.astype(int)

In [157]:
merged.head()

,time,mid_h,mid_l,start_price,SIGNAL,TP,SL,start_index_h1
0,2016-01-03 22:00:00+00:00,1.08746,1.08724,0.0,0,0.0,0.0,0
1,2016-01-03 22:05:00+00:00,1.08738,1.08710,0.0,0,0.0,0.0,0
2,2016-01-03 22:10:00+00:00,1.08715,1.08674,0.0,0,0.0,0.0,0
3,2016-01-03 22:15:00+00:00,1.08694,1.08669,0.0,0,0.0,0.0,0
4,2016-01-03 22:20:00+00:00,1.08717,1.08670,0.0,0,0.0,0.0,0


In [166]:
class TradeM5:
    def __init__(self, row):
        self.running = True
        self.start_index_m5 = row.name
        self.start_index_h1 = row.start_index_h1
        self.start_price = row.start_price
        self.trigger_price = row.start_price
        self.SIGNAL = row.SIGNAL
        self.TP = row.TP
        self.SL = row.SL
        self.result = 0.0
        self.end_time = row.time
        self.start_time = row.time
        self.duration = 1
    
    def close_trade(self, row, result, trigger_price):
        self.running = False
        self.result = result
        self.end_time = row.time
        self.trigger_price = trigger_price
    
    def update(self, row):
        self.duration += 1
        if self.SIGNAL == BUY:
            if row.mid_h >= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.mid_h)
            elif row.mid_l <= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.mid_l)
        if self.SIGNAL == SELL:
            if row.mid_l <= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.mid_l)
            elif row.mid_h >= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.mid_h)
    

In [167]:
open_trades_m5 = []
closed_trades_m5 = []

for index, row in merged.iterrows():
    if row.SIGNAL != NONE:
        open_trades_m5.append(TradeM5(row))
        
    for ot in open_trades_m5:
        ot.update(row)
        if ot.running == False:
            closed_trades_m5.append(ot)
    open_trades_m5 = [x for x in open_trades_m5 if x.running == True]

In [168]:
df_res_m5 = pd.DataFrame.from_dict([vars(x) for x in closed_trades_m5])

In [169]:
df_res_m5.head()

,running,start_index_m5,start_index_h1,start_price,trigger_price,SIGNAL,TP,SL,result,end_time,start_time,duration
0,False,2459,5,1.09227,1.08852,1,1.095390,1.09019,-1.0,2016-01-14 12:35:00+00:00,2016-01-14 11:00:00+00:00,21
1,False,2435,3,1.09134,1.08564,1,1.099440,1.08594,-1.0,2016-01-14 15:40:00+00:00,2016-01-14 09:00:00+00:00,82
2,False,2699,25,1.08872,1.08710,1,1.089980,1.08788,-1.0,2016-01-15 07:30:00+00:00,2016-01-15 07:00:00+00:00,8
3,False,2723,27,1.09007,1.09567,1,1.094900,1.08685,1.5,2016-01-15 13:35:00+00:00,2016-01-15 09:00:00+00:00,57
4,False,2819,35,1.09635,1.09834,1,1.098225,1.09510,1.5,2016-01-15 17:25:00+00:00,2016-01-15 17:00:00+00:00,7


In [170]:
df_res_m5.result.sum()

-76.5